In [2]:
#https://towardsdatascience.com/programmatic-identification-of-support-resistance-trend-lines-with-python-d797a4a90530

In [12]:
import pandas as pd
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [25]:
df = yf.Ticker('PLTR').history(period='1y', interval='1d')

tick = yf.Ticker('^GSPC')
hist = tick.history(period="max", rounding=True)
# hist = hist[:'2019-10-07']
hist = hist[-1000:]
h = hist.Close.tolist()

In [26]:
fig = go.Figure(data=[go.Candlestick(x=df.index, open=df.Open, close=df.Close, high=df.High, low=df.Low)])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [27]:
fig = go.Figure(data=[go.Candlestick(x=hist.index, open=hist.Open, close=hist.Close, high=hist.High, low=hist.Low)])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [21]:
from findiff import FinDiff

In [22]:
dx = 1
d_dx = FinDiff(0, dx, 1)
d2_dx2 = FinDiff(0, dx, 2)
clarr = np.asarray(hist.Close)
mom = d_dx(clarr)
momacc = d2_dx2(clarr)

In [24]:
def get_extrema(isMin):
    return [x for x in range(len(mom))
        if (momacc[x] > 0 if isMin else momacc[x] < 0) and
        (mom[x] == 0 or #slope is 0
            (x != len(mom) - 1 and #check next day
                (mom[x] > 0 and mom[x+1] < 0 and
                h[x] >= h[x+1] or
                mom[x] < 0 and mom[x+1] > 0 and
                h[x] <= h[x+1]) or
            x != 0 and #check prior day
                (mom[x-1] > 0 and mom[x] < 0 and
                h[x-1] < h[x] or
                mom[x-1] < 0 and mom[x] > 0 and
                h[x-1] > h[x])))]
minimaIdxs, maximaIdxs = get_extrema(True), get_extrema(False)